# Vehicle routing with Timefold in a Kotlin notebook

In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.5.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.16.1")

## Domain

Create the domain data classes:

### Location

In [2]:
data class Location(
    val latitude: Double,
    val longitude: Double)

### Visit

In [3]:
import java.time.Duration

data class Visit(

    val name: String,
    val location: Location,
    val demand: Int) {

    override fun toString(): String = name
}

### Vehicle

In [4]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable


@PlanningEntity
data class Vehicle(

    val name: String,
    val homeLocation: Location,
    val capacity: Int) {

    @PlanningVariable
    var visits: List<Visit>? = null

    override fun toString(): String = name
}

## Constraints

In [5]:
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class VehicleRoutingConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            // Soft constraints
        )
    }

}

## Schedule

In [6]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import kotlinx.serialization.Transient


@PlanningSolution
data class Schedule(
    
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>) {

    @PlanningScore
    @Transient
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList())

}

## Data generator

In [13]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()
val problem: Schedule =  mapper.readValue(File("vehicle-routing-data.json"))



problem

Schedule(vehicles=[Vehicle 1], visits=[Anthisnes])

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).